# RAG System for Historical Tourism Q&A

Goal: Build an end-to-end RAG pipeline integrating MySQL (structured data), Qdrant (vector DB), and an LLM generator. The system returns cited, grounded answers for history + travel queries.

Stack: Python, MySQL, Qdrant, Sentence-Transformers embeddings, OpenAI (optional) for generation.


In [ ]:
# Setup: installs (skip if already installed)
%pip -q install -r requirements.txt

import os
from dotenv import load_dotenv
load_dotenv(".env", override=True)
load_dotenv("ENV.example", override=False)

MYSQL_HOST = os.getenv("MYSQL_HOST", "localhost")
MYSQL_PORT = int(os.getenv("MYSQL_PORT", 3306))
MYSQL_USER = os.getenv("MYSQL_USER", "root")
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD", "")
MYSQL_DATABASE = os.getenv("MYSQL_DATABASE", "rag_history_tourism")

QDRANT_URL = os.getenv("QDRANT_URL", "http://localhost:6333")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
QDRANT_COLLECTION = os.getenv("QDRANT_COLLECTION", "rag_history_tourism")

EMBEDDING_MODEL = os.getenv("EMBEDDING_MODEL", "sentence-transformers/all-MiniLM-L6-v2")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")

print({
    "mysql": f"{MYSQL_USER}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DATABASE}",
    "qdrant": QDRANT_URL,
    "collection": QDRANT_COLLECTION,
    "embedding_model": EMBEDDING_MODEL,
    "openai_model": OPENAI_MODEL,
})


In [ ]:
# Connect to MySQL and create schema + seed data
import mysql.connector as mysql
import pandas as pd

conn = mysql.connect(
    host=MYSQL_HOST,
    port=MYSQL_PORT,
    user=MYSQL_USER,
    password=MYSQL_PASSWORD,
)
conn.autocommit = True
cur = conn.cursor()
cur.execute(f"CREATE DATABASE IF NOT EXISTS {MYSQL_DATABASE} CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;")
cur.close()
conn.close()

conn = mysql.connect(
    host=MYSQL_HOST,
    port=MYSQL_PORT,
    user=MYSQL_USER,
    password=MYSQL_PASSWORD,
    database=MYSQL_DATABASE,
)
cur = conn.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS places (
  id INT AUTO_INCREMENT PRIMARY KEY,
  name VARCHAR(255) NOT NULL,
  country VARCHAR(128),
  city VARCHAR(128),
  lat DOUBLE,
  lon DOUBLE,
  description TEXT,
  url VARCHAR(512)
) ENGINE=InnoDB;
""")
cur.execute("""
CREATE TABLE IF NOT EXISTS events (
  id INT AUTO_INCREMENT PRIMARY KEY,
  place_id INT,
  title VARCHAR(255) NOT NULL,
  year_start INT,
  year_end INT,
  summary TEXT,
  source VARCHAR(512),
  FOREIGN KEY (place_id) REFERENCES places(id) ON DELETE SET NULL
) ENGINE=InnoDB;
""")

# Minimal seed data: can expand later
places = [
    ("Hoan Kiem Lake", "Vietnam", "Hanoi", 21.0285, 105.8542, "Historic lake in the Old Quarter, legend of the returned sword.", "https://en.wikipedia.org/wiki/Ho%C3%A0n_Ki%E1%BA%BFm_Lake"),
    ("Hue Imperial City", "Vietnam", "Hue", 16.4637, 107.5909, "Former imperial capital with Nguyen dynasty citadel.", "https://en.wikipedia.org/wiki/Imperial_City_of_Hu%E1%BA%BF"),
]
cur.executemany(
    "INSERT INTO places (name, country, city, lat, lon, description, url) VALUES (%s,%s,%s,%s,%s,%s,%s)",
    places,
)

cur.execute("SELECT id, name FROM places")
place_map = {name: pid for (pid, name) in cur.fetchall()}

events = [
    (place_map["Hoan Kiem Lake"], "Legend of the Returned Sword", 1428, 1428, "L0fi thb0db0 Tra nea Hung Da o legend: Emperor La Le i returned the divine sword to the Golden Turtle God at the lake.", "https://en.wikipedia.org/wiki/Ho%C3%A0n_Ki%E1%BA%BFm_Lake"),
    (place_map["Hue Imperial City"], "Nguyen Dynasty Capital", 1802, 1945, "Hue served as the imperial capital of the Nguyen dynasty, with significant architectural and cultural heritage.", "https://en.wikipedia.org/wiki/Imperial_City_of_Hu%E1%BA%BF"),
]
cur.executemany(
    "INSERT INTO events (place_id, title, year_start, year_end, summary, source) VALUES (%s,%s,%s,%s,%s,%s)",
    events,
)
conn.commit()

cur.close()
conn.close()

print("Seeded MySQL with sample places and events.")


In [ ]:
# Build embeddings and ingest into Qdrant
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http import models as qmodels

# Fetch documents from MySQL
conn = mysql.connect(
    host=MYSQL_HOST,
    port=MYSQL_PORT,
    user=MYSQL_USER,
    password=MYSQL_PASSWORD,
    database=MYSQL_DATABASE,
)
cur = conn.cursor(dictionary=True)
cur.execute("""
SELECT e.id AS event_id, p.id AS place_id, p.name AS place_name, p.country, p.city, p.lat, p.lon,
       e.title, e.year_start, e.year_end, e.summary, COALESCE(e.source, p.url) AS source
FROM events e LEFT JOIN places p ON e.place_id = p.id
ORDER BY e.id ASC
""")
rows = cur.fetchall()
cur.close(); conn.close()

# Prepare texts
def format_doc(r):
    yrs = f"{r['year_start']}" if r['year_end'] in (None, 0, r['year_start']) else f"{r['year_start']}–{r['year_end']}"
    loc = ", ".join([x for x in [r['place_name'], r['city'], r['country']] if x])
    return f"{r['title']} ({yrs}) at {loc}. {r['summary']}"

texts = [format_doc(r) for r in rows]
metadatas = rows

# Embeddings
embedder = SentenceTransformer(EMBEDDING_MODEL)
embeddings = embedder.encode(texts, normalize_embeddings=True)

# Qdrant client and collection
qdrant = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY)

if QDRANT_COLLECTION not in [c.name for c in qdrant.get_collections().collections]:
    qdrant.recreate_collection(
        collection_name=QDRANT_COLLECTION,
        vectors_config=qmodels.VectorParams(size=len(embeddings[0]), distance=qmodels.Distance.COSINE),
    )

# Upsert points
from uuid import uuid4
payloads = metadatas
qdrant.upsert(
    collection_name=QDRANT_COLLECTION,
    points=[
        qmodels.PointStruct(
            id=str(uuid4()),
            vector=emb.tolist(),
            payload=payload,
        )
        for emb, payload in zip(embeddings, payloads)
    ],
)

print(f"Ingested {len(payloads)} documents into Qdrant collection '{QDRANT_COLLECTION}'.")


In [ ]:
# Retriever and generator with citations
from typing import List, Dict

TOP_K = 5


def retrieve(query: str, k: int = TOP_K) -> List[Dict]:
    q_emb = embedder.encode([query], normalize_embeddings=True)[0]
    search = qdrant.search(
        collection_name=QDRANT_COLLECTION,
        query_vector=q_emb.tolist(),
        limit=k,
        with_payload=True,
        with_vectors=False,
        score_threshold=None,
    )
    docs = []
    for point in search:
        payload = point.payload or {}
        payload["score"] = float(point.score)
        docs.append(payload)
    return docs


def render_context(docs: List[Dict]) -> str:
    lines = []
    for d in docs:
        title = d.get("title") or d.get("place_name")
        source = d.get("source") or d.get("url") or ""
        city = d.get("city") or ""
        country = d.get("country") or ""
        yrs = f"{d.get('year_start')}" if (not d.get('year_end') or d.get('year_end') == d.get('year_start')) else f"{d.get('year_start')}–{d.get('year_end')}"
        line = f"- {title} ({yrs}), {city}, {country}. Source: {source}"
        lines.append(line)
    return "\n".join(lines)


# Simple generation using OpenAI if available, else a rule-based fallback
try:
    from openai import OpenAI
    openai_client = OpenAI(api_key=OPENAI_API_KEY) if OPENAI_API_KEY else None
except Exception:
    openai_client = None


def generate_answer(query: str, docs: List[Dict]) -> str:
    context = "\n".join(
        [
            f"Title: {d.get('title')} | Place: {d.get('place_name')}, {d.get('city')}, {d.get('country')} | Years: {d.get('year_start')}-{d.get('year_end')}\nSummary: {d.get('summary')}\nSource: {d.get('source') or d.get('url') or ''}"
            for d in docs
        ]
    )
    system = (
        "You are a factual travel history assistant. Use only the provided CONTEXT to answer. "
        "Cite sources inline as [n] and add a 'Sources' section mapping [n] to URLs. If the answer is not in context, say you don't know."
    )

    if openai_client:
        messages = [
            {"role": "system", "content": system},
            {"role": "user", "content": f"CONTEXT:\n{context}\n\nQUESTION: {query}"},
        ]
        try:
            resp = openai_client.chat.completions.create(
                model=OPENAI_MODEL,
                messages=messages,
                temperature=0.2,
            )
            return resp.choices[0].message.content
        except Exception as e:
            print("OpenAI error:", e)

    # Fallback: extractive style summary
    answer_lines = [
        "Based on available context:",
    ]
    for i, d in enumerate(docs, start=1):
        src = d.get("source") or d.get("url") or ""
        yrs = f"{d.get('year_start')}" if (not d.get('year_end') or d.get('year_end') == d.get('year_start')) else f"{d.get('year_start')}-{d.get('year_end')}"
        answer_lines.append(
            f"[{i}] {d.get('title')} in {d.get('place_name')}, {d.get('city')}, {d.get('country')} ({yrs}): {d.get('summary')}"
        )
        if src:
            answer_lines.append(f"Source: {src}")
    if not docs:
        answer_lines.append("No matching context found.")
    return "\n".join(answer_lines)


# Convenience wrapper

def ask(query: str, k: int = 5) -> str:
    docs = retrieve(query, k=k)
    print("Top documents:\n" + render_context(docs))
    print("\n---\n")
    return generate_answer(query, docs)


In [ ]:
# Demo
query = "Lịch sử và gợi ý tham quan quanh Hồ Hoàn Kiếm?"
print(ask(query, k=3))
